# Load as LUME-model for use with EPICS data

Loads the base neural network model, the `sim_to_nn` & `pv_to_sim` transformers and the PV variable specification from their respective files to create a [LUME-model](https://github.com/slaclab/lume-model/). The resulting instance of `TorchModel` enforces requirements on the input and output variables and can be wrapped in a `TorchModule`. The `TorchModule` can be used like a `torch.nn.Module` and is tested on a small set of data.

When working with EPICS PV data, it's important to remember that the data you pull from the archive will require some post-processing before it can be used with the surrogate model below. 

For example:
* outliers may need to be removed
* unphysical values may need to be removed (e.g. XRMS and YRMS values below 0)
* values for features that lie outside the training distribution should be removed
* values for unmeasurable PVs (such as the pulse length) will need to be imputed (`Pulse_length`)
* values for features that were not varied in the training data will need to be overwritten with the PV unit equivalent of the value used during training (`FBCK:BCI0:1:CHRG_S` and `ACCL:IN20:400:L0B_ADES`)
* compound PVs (PVs created from a calculation using measured PVs) will need to be calculated (`CAMR:IN20:186:R_DIST`)

`CAMR:IN20:186:R_DIST` is calculated from `CAMR:IN20:186:XRMS` and `CAMR:IN20:186:YRMS` using the following formula:
```python
r_dist = np.sqrt(data["CAMR:IN20:186:XRMS"].values ** 2 + data["CAMR:IN20:186:YRMS"].values ** 2)
```

Further information about the specific changes that need to be applied for this model can be found in the [README](README.md) and the [PV variable specification](model/pv_variables.yml). 

In [1]:
import torch
import matplotlib.pyplot as plt
import os
import mlflow

from lume_model.utils import variables_from_yaml
from lume_model.models import TorchModel, TorchModule

In [2]:
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env")         

True

In [3]:
mlflow.set_tracking_uri(os.environ['MLFLOW_TRACKING_URI'])
mlflow.set_experiment("lcls-injector-ML")

<Experiment: artifact_location='s3://mlflow-bucket/5', creation_time=1746036982208, experiment_id='5', last_update_time=1746036982208, lifecycle_stage='active', name='lcls-injector-ML', tags={}>

In [4]:
if mlflow.active_run():
    mlflow.end_run()

mlflow.start_run(run_name="lume_model_epics Run2")

2025/05/02 10:16:19 INFO mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics. Set logger level to DEBUG for more details.
2025/05/02 10:16:19 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


<ActiveRun: >

## Create model

In [5]:
# load sim_to_nn transformers
input_sim_to_nn = torch.load("../model/input_sim_to_nn.pt")
output_sim_to_nn = torch.load("../model/output_sim_to_nn.pt")

# load pv_to_sim transformers
input_pv_to_sim = torch.load('../model/input_pv_to_sim.pt')
output_pv_to_sim = torch.load('../model/output_pv_to_sim.pt')

mlflow.log_param("input_transformers", f"{type(input_pv_to_sim).__name__} -> {type(input_sim_to_nn).__name__}")
mlflow.log_param("output_transformers", f"{type(output_sim_to_nn).__name__} -> {type(output_pv_to_sim).__name__}")

'AffineInputTransform -> AffineInputTransform'

In [6]:
# load in- and output variable specification
input_variables, output_variables = variables_from_yaml("../model/pv_variables.yml")

mlflow.log_param("input_variables", input_variables)
mlflow.log_param("output_variables", output_variables)
mlflow.log_artifact("../model/pv_variables.yml")

In [7]:
# create TorchModel
lume_model = TorchModel(
    model="../model/model.pt",
    input_variables=input_variables,
    output_variables=output_variables,
    input_transformers=[input_pv_to_sim, input_sim_to_nn],  # pv_to_sim before sim_to_nn
    output_transformers=[output_sim_to_nn, output_pv_to_sim],  # sim_to_nn before pv_to_sim
)

# or simply load from YAML file
lume_model = TorchModel("../model/pv_model.yml")

Loaded PyTorch model from file: ../model/model.pt
Loaded PyTorch model from file: /sdf/home/g/gopikab/lcls-ml/lcls_cu_injector_ml_model/model/model.pt


In [8]:
# wrap in TorchModule
lume_module = TorchModule(
    model=lume_model,
    input_order=lume_model.input_names,
    output_order=lume_model.output_names,
)

# or simply load from YAML file
lume_module = TorchModule("../model/pv_module.yml")

mlflow.log_artifact("../model/pv_module.yml")

Loaded PyTorch model from file: /sdf/home/g/gopikab/lcls-ml/lcls_cu_injector_ml_model/model/model.pt


## Test on example data

In [9]:
# load example data and calculate predictions
inputs_small = input_pv_to_sim.untransform(torch.load("../info/inputs_small.pt"))
outputs_small = output_pv_to_sim.untransform(torch.load("../info/outputs_small.pt"))
with torch.no_grad():
    predictions = lume_module(inputs_small)
# Log performance metric
mae = torch.mean(torch.abs(predictions - outputs_small)).item()
mlflow.log_metric("mean_absolute_error", mae)

In [10]:
# plot example data and predictions
nrows, ncols = 3, 2
fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(10, 15))
for i, output_name in enumerate(lume_module.output_order):
    ax_i = ax[i // ncols, i % ncols]
    if i < outputs_small.shape[1]:
        sort_idx = torch.argsort(outputs_small[:, i])
        x_axis = torch.arange(outputs_small.shape[0])
        ax_i.plot(x_axis, outputs_small[sort_idx, i], "C0x", label="outputs")
        ax_i.plot(x_axis, predictions[sort_idx, i], "C1x", label="predictions")
        ax_i.legend()
        ax_i.set_title(output_name)
ax[-1, -1].axis('off')
fig.tight_layout()


plot_path = "epics_plot_lume.png"
plt.savefig(plot_path)
mlflow.log_artifact(plot_path)
plt.close()

## Missing Output PVs

If we have missing outputs, for example outputs of the model that are not measured in EPICS, we can simply remove these from the `TorchModule` by passing a truncated list to `output_order` (they still need to be listed in the TorchModel though).

In [11]:
# wrap in truncated TorchModule
truncated_lume_module = TorchModule(
    model=lume_model,
    input_order=lume_model.input_names,
    output_order=lume_model.output_names[0:2],  # truncate list of parameters
)
truncated_lume_module.output_order

['OTRS:IN20:571:XRMS', 'OTRS:IN20:571:YRMS']

In [13]:
# calculate predictions
with torch.no_grad():
    predictions = truncated_lume_module(inputs_small)
predictions.shape

torch.Size([283, 2])

In [14]:
# plot example data and predictions
nrows, ncols = 1, 2
fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(10, 5))
for i, output_name in enumerate(truncated_lume_module.output_order):
    sort_idx = torch.argsort(outputs_small[:, i])
    x_axis = torch.arange(outputs_small.shape[0])
    ax[i].plot(x_axis, outputs_small[sort_idx, i], "C0x", label="outputs")
    ax[i].plot(x_axis, predictions[sort_idx, i], "C1x", label="predictions")
    ax[i].legend()
    ax[i].set_title(output_name)
fig.tight_layout()


plot_path = "epics_lume.png"
plt.savefig(plot_path)
mlflow.log_artifact(plot_path)
plt.close()

In [15]:
if mlflow.active_run():
    mlflow.end_run()

2025/05/02 10:37:05 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...


🏃 View run lume_model_epics Run2 at: https://ard-mlflow.slac.stanford.edu/#/experiments/5/runs/96458aef9537417d92e69e33d74bf777
🧪 View experiment at: https://ard-mlflow.slac.stanford.edu/#/experiments/5


2025/05/02 10:37:05 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
